In [2]:
# necessary libraries that I thought
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential

# Here reading the data set, I am reading both here only
train_data = pd.read_csv('Train.csv')
test_data = pd.read_csv('Test.csv')


#some little preprocessing
#I creted the lebels "Text" and "Sentiment" as it was not in the data set
X_train = train_data['Text'].astype(str)
X_test = test_data['Text'].astype(str)

# As computers dont undertand words, converting it into numbers               #note I tried first with words:p
label_mapping = {'Positive': 1, 'Negative': 0, 'Neutral': 2, 'Irrelevant': 3}
y_train = train_data['Sentiment'].map(label_mapping)
y_test = test_data['Sentiment'].map(label_mapping)

# TF-IDF learnt in NLP, I liked this
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Simply taken Naive bayes fucntion from sklearn
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train_tfidf, y_train)

# Evaluation thing I checked
y_pred_nb = nb_classifier.predict(X_test_tfidf)
accuracy_nb = accuracy_score(y_test, y_pred_nb)
precision_nb = precision_score(y_test, y_pred_nb, average='weighted')
recall_nb = recall_score(y_test, y_pred_nb, average='weighted')
f1_nb = f1_score(y_test, y_pred_nb, average='weighted')

# LSTM model, took help
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)
X_train_pad = pad_sequences(X_train_seq, maxlen=200)
X_test_pad = pad_sequences(X_test_seq, maxlen=200)

model = Sequential([
    Embedding(input_dim=5000, output_dim=64, input_length=200),
    LSTM(64),
    Dense(4, activation='softmax')  # softmax for LSTM
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train_pad, y_train, epochs=5, batch_size=32, validation_data=(X_test_pad, y_test))

# Evaluatoin kinda same as above checked the difference
y_pred_dl_prob = model.predict(X_test_pad)
y_pred_dl = y_pred_dl_prob.argmax(axis=-1)
accuracy_dl = accuracy_score(y_test, y_pred_dl)
precision_dl = precision_score(y_test, y_pred_dl, average='weighted')
recall_dl = recall_score(y_test, y_pred_dl, average='weighted')
f1_dl = f1_score(y_test, y_pred_dl, average='weighted')


new_text = "I am sad"    #took a text, real life incident


#prediction work
new_text_tfidf = tfidf_vectorizer.transform([new_text])
predicted_sentiment_nb = nb_classifier.predict(new_text_tfidf)



new_text_seq = tokenizer.texts_to_sequences([new_text])
new_text_pad = pad_sequences(new_text_seq, maxlen=200)
predicted_sentiment_dl_prob = model.predict(new_text_pad)
predicted_sentiment_dl = predicted_sentiment_dl_prob.argmax(axis=-1)


Epoch 1/5
2334/2334 [==============================] - 86s 35ms/step - loss: 0.9277 - accuracy: 0.6210 - val_loss: 0.5345 - val_accuracy: 0.8150
Epoch 2/5
2334/2334 [==============================] - 33s 14ms/step - loss: 0.6328 - accuracy: 0.7548 - val_loss: 0.4026 - val_accuracy: 0.8760
Epoch 3/5
2334/2334 [==============================] - 37s 16ms/step - loss: 0.4853 - accuracy: 0.8152 - val_loss: 0.3645 - val_accuracy: 0.8940
Epoch 4/5
2334/2334 [==============================] - 26s 11ms/step - loss: 0.3938 - accuracy: 0.8494 - val_loss: 0.3674 - val_accuracy: 0.8990
Epoch 5/5
1/1 [==============================] - 0s 43ms/step


In [9]:
print("Deep Learning model's accuracy:" ,accuracy_dl)
print("Deep Learning model's precision:" ,precision_dl)
print("Deep Learning model's recall:" ,recall_dl )
print("Deep Learning model's F1 Score:" ,f1_dl )

Deep Learning model's accuracy: 0.907
Deep Learning model's precision: 0.9072515349341801
Deep Learning model's recall: 0.907
Deep Learning model's F1 Score: 0.9067674919074159


In [10]:
print("Machine  Learning model's accuracy:" ,accuracy_nb)
print("Machine Learning model's precision:" ,precision_nb)
print("Machine Learning model's recall:" ,recall_nb )
print("Machine Learning model's F1 Score:" ,f1_nb )

Machine  Learning model's accuracy: 0.72
Machine Learning model's precision: 0.7361955141435995
Machine Learning model's recall: 0.72
Machine Learning model's F1 Score: 0.7157218066728582


In [5]:
print(predicted_sentiment_dl)    #this also workd, as it is giving the label assigned: "Negative = 0"

[0]


In [3]:
predicted_label = []                                   # But if the want the Sentiment
for i in range(0,len(predicted_sentiment_dl)):
  if predicted_sentiment_dl[i] in label_mapping.values():
    for j in (list(label_mapping.keys())):
      # print(i)
      if label_mapping[j] == predicted_sentiment_dl[i]:
        predicted_label.append(j)

In [4]:
predicted_label

['Negative']